In [2]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset

In [5]:
datasetss = load_dataset("multi_nli").data["train"]

In [7]:
datasetss = [
    {
        "premise": str(p),
        "hypothesis": str(h),
        "labels": l.as_py(),
    }
    for p, h, l in zip(datasetss[2], datasetss[5], datasetss[9])
]

In [8]:
#datasetss

In [28]:
raw_datasets = load_dataset("multi_nli")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})

In [8]:
train_datasets = datasets["train"]
train_datasets[:2]

{'promptID': [31193, 101457],
 'pairID': ['31193n', '101457e'],
 'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.',
  'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him'],
 'premise_binary_parse': ['( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )',
  '( you ( ( know ( during ( ( ( the season ) and ) ( i guess ) ) ) ) ( at ( at ( ( your level ) ( uh ( you ( ( ( lose them ) ( to ( the ( next level ) ) ) ) ( if ( ( if ( they ( decide ( to ( recall ( the ( the ( parent team ) ) ) ) ) ) ) ) ( ( the Braves ) ( decide ( to ( call ( to ( ( recall ( a guy ) ) ( from ( ( triple A ) ( ( ( then ( ( a ( double ( A guy ) ) ) ( ( goes up ) ( to ( replace him )

In [11]:
"""
datasets = [
    {
        "premise": p,
        "hypothesis": h,
        "labels": l,
    }
    for p, h, l in zip(datasets['train']["premise"], datasets['train']["hypothesis"], datasets['train']["label"] )
]
"""

In [12]:
#datasets

In [13]:
#data_loader = DataLoader(datasets, batch_size=128, num_workers=4)

In [14]:
#!pip show transformers

Name: transformers
Version: 4.40.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /scratch/qualis/miniconda3/envs/dp/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [6]:
train_datasets.features

{'promptID': Value(dtype='int32', id=None),
 'pairID': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'premise_binary_parse': Value(dtype='string', id=None),
 'premise_parse': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'hypothesis_binary_parse': Value(dtype='string', id=None),
 'hypothesis_parse': Value(dtype='string', id=None),
 'genre': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [9]:
from transformers import AutoTokenizer

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
input = tokenizer(train_datasets["premise"][0],
                  train_datasets["hypothesis"][0])
input

{'input_ids': [101, 28103, 14795, 7081, 10458, 25004, 1144, 1160, 3501, 10082, 118, 3317, 1105, 14534, 119, 102, 22249, 1105, 14534, 1132, 1184, 1294, 7081, 10458, 25004, 1250, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
input.tokens()

['[CLS]',
 'Concept',
 '##ually',
 'cream',
 'ski',
 '##mming',
 'has',
 'two',
 'basic',
 'dimensions',
 '-',
 'product',
 'and',
 'geography',
 '.',
 '[SEP]',
 'Product',
 'and',
 'geography',
 'are',
 'what',
 'make',
 'cream',
 'ski',
 '##mming',
 'work',
 '.',
 '[SEP]']

In [15]:
tokenized_dataset = tokenizer(
    train_datasets["premise"],
    train_datasets["hypothesis"],
    padding=True,
    truncation=True,
)

In [17]:
tokenized_dataset[:2]

[Encoding(num_tokens=444, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=444, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [18]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

In [19]:
tokenized_datasets = train_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 392702
})

In [22]:
#tokenized_datasets[:2]

In [21]:
samples = tokenized_datasets[:8]

samples = {k: v for k, v in samples.items() if k not in ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre']}
[len(x) for x in samples["input_ids"]]

KeyError: "Invalid key: slice(None, 8, None). Please first select a split. For example: `my_dataset_dictionary['train'][slice(None, 8, None)]`. Available splits: ['test', 'train']"

In [25]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78]),
 'labels': torch.Size([8])}

In [10]:
#batch

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
train_ds =  load_dataset("multi_nli", split="train")

In [4]:
train_ds

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 392702
})

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 353431
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39271
    })
})

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("multi_nli")

train_raw_datasets = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = train_raw_datasets.map(tokenize_function, batched=True)
#tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [32]:
tokenized_datasets["train"].shuffle(seed=12).select(range(5000))

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

In [35]:
#!pip install accelerate -U

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "multi_nli-test-trainer",
    max_steps = 5000
)

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
#print(model)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [34]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.36911417643229166, metrics={'train_runtime': 14.3444, 'train_samples_per_second': 209.141, 'train_steps_per_second': 26.143, 'total_flos': 112789322115120.0, 'train_loss': 0.36911417643229166, 'epoch': 3.0})

In [23]:
predictions = trainer.predict(tokenized_datasets["test"])
#print(predictions.predictions.shape, predictions.label_ids.shape)

In [24]:
print(predictions.predictions.shape, predictions.label_ids.shape)
print(predictions)

(39271, 3) (39271,)
PredictionOutput(predictions=array([[ 0.65015644, -1.049954  , -0.16636048],
       [ 1.7663594 , -1.4315085 , -1.1739348 ],
       [-0.25628814,  0.13601816, -0.42100936],
       ...,
       [ 1.0100716 , -0.7710598 , -0.7481078 ],
       [-1.5408589 ,  2.5510836 , -1.3346443 ],
       [-2.3487828 ,  0.5423903 ,  1.5136144 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 2]), metrics={'test_loss': 0.651908814907074, 'test_runtime': 52.9439, 'test_samples_per_second': 741.747, 'test_steps_per_second': 92.721})


In [25]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [53]:
#!pip install evaluate

In [ ]:
#!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [38]:
import evaluate
metric = evaluate.load('accuracy')
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7360647806269257}

In [49]:
import evaluate
metric = evaluate.load('accuracy')
#clf_metric = evaluate.combine(["accuracy", "f1", "precision", "recall"]) # working with binary classification
 
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [56]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir = "multi_nli-test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    #max_steps = 5000
)

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=12).select(range(5000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=12).select(range(500)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.048700,2.971684,0.656000
2,0.078200,2.546162,0.678000
3,0.044400,2.837656,0.688000


TrainOutput(global_step=1875, training_loss=0.05281803843180338, metrics={'train_runtime': 73.2059, 'train_samples_per_second': 204.902, 'train_steps_per_second': 25.613, 'total_flos': 560657692854000.0, 'train_loss': 0.05281803843180338, 'epoch': 3.0})

### Full Training

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("multi_nli", split="train")

train_test_datasets = raw_datasets.train_test_split(test_size=0.1, seed=42)

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = train_test_datasets.map(tokenize_function, batched=True)
#tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [2]:
tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 353431
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39271
    })
})

In [3]:
from torch.utils.data import DataLoader

train_datasets = tokenized_datasets["train"].shuffle(seed=12).select(range(20000))
valid_datasets = tokenized_datasets["test"].shuffle(seed=12).select(range(2000))


train_dataloader = DataLoader(
    #tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
    train_datasets, shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    #tokenized_datasets["test"], batch_size=64, collate_fn=data_collator
    valid_datasets, batch_size=64, collate_fn=data_collator
)

In [4]:
#for batch in train_dataloader:
#    break
#{k: v.shape for k, v in batch.items()}

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#outputs = model(**batch)
#print(outputs.loss, outputs.logits.shape)

In [7]:
#print(outputs)

In [8]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/scratch/qualis/miniconda3/envs/dp/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1875


In [10]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [11]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1875 [00:00<?, ?it/s]

In [12]:
import evaluate
metric = evaluate.load("accuracy")

num_eval_steps = len(eval_dataloader)
progress_bar = tqdm(range(num_eval_steps))

model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar.update(1)

metric.compute()

  0%|          | 0/32 [00:00<?, ?it/s]

{'accuracy': 0.741}

In [16]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("multi_nli", split="train")

train_test_datasets = raw_datasets.train_test_split(test_size=0.1, seed=42)

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = train_test_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from torch.utils.data import DataLoader

train_datasets = tokenized_datasets["train"].shuffle(seed=12).select(range(20000))
valid_datasets = tokenized_datasets["test"].shuffle(seed=12).select(range(2000))



train_dataloader = DataLoader(
    #tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
    train_datasets, shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    #tokenized_datasets["test"], batch_size=64, collate_fn=data_collator
    valid_datasets, batch_size=64, collate_fn=data_collator
)

from transformers import AutoModelForSequenceClassification
from transformers import AdamW

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}
        batch = batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)



import evaluate
metric = evaluate.load("accuracy")

num_eval_steps = len(eval_dataloader)
progress_bar = tqdm(range(num_eval_steps))

model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar.update(1)

metric.compute()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/scratch/qualis/miniconda3/envs/transformer/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

{'accuracy': 0.74}

### Accelerator

In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("multi_nli", split="train")

train_test_datasets = raw_datasets.train_test_split(test_size=0.1, seed=42)

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = train_test_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from torch.utils.data import DataLoader

train_datasets = tokenized_datasets["train"].shuffle(seed=12).select(range(20000))
valid_datasets = tokenized_datasets["test"].shuffle(seed=12).select(range(2000))



train_dataloader = DataLoader(
    #tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
    train_datasets, shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    #tokenized_datasets["test"], batch_size=64, collate_fn=data_collator
    valid_datasets, batch_size=64, collate_fn=data_collator
)

from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from tqdm.auto import tqdm

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
optimizer = AdamW(model.parameters(), lr=3e-5)

#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#model.to(device)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        #loss.backward()
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

import evaluate

metric = evaluate.load("accuracy")

model.eval()
for batch in eval_dl:
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# 평가 결과 계산 및 출력 
metric.compute()


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1875 [00:00<?, ?it/s]

{'accuracy': 0.742}

In [19]:

loss=torch.tensor([1.1841, 1.2422, 1.2407, 1.0912])

In [22]:
torch.mean(loss).item()

1.1895500421524048